In [2]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import yfinance as yf
import numpy as np
import pyfolio as pf
from io import StringIO
import sklearn as skn
from scipy import stats
import pandas_datareader.data as pdr


c:\Users\matsz\miniconda3\envs\portfolio-optimisation\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [8]:
def loadStooqData(ticker: str,start, frequency='d'):
    url = f'https://stooq.pl/q/d/l/?s={ticker}&i=d'
    response = requests.get(url)  
    data = pd.read_csv(StringIO(response.text))
    data.set_index('Data', inplace=True)
    data = data[start:]
    return data['Zamkniecie']

def loadYahooData(ticker, start, frequency = '1d'):      
    ticker  = yf.Ticker(ticker)
    hist = ticker.history(start=start,interval=frequency)
    price = hist['Close']       
    return price

sharp = lambda returns: returns.mean()/returns.std()

def assets_performance(returns: pd.DataFrame):
    return returns.agg(['mean', 'std', 'median', 'skew', 'kurtosis', sharp])

def beta(X, Y):
       
    X_cov = X.cov()

    XY = np.append(X,np.expand_dims(Y,axis=1),axis=0)
    XY_cov = np.cov(XY)

    #XY_sub = XY_cov.iloc[-1,0:2]

    #inv = np.linalg.inv(X_cov)
    #B = inv @ XY_sub
    return  XY_cov

In [4]:
yahoo_tickers = ['ISAC.L', 'CORP.L']

stooq_tickers = ['XAUPLN', '^TBSP', 'MWIG40TR', 'USDPLN', 'EURPLN']

#na przyszłość EAFA,EEM,ACWI



In [19]:
base_prices = pd.DataFrame({})
start = '2012-01-01'

for ticker in stooq_tickers:
    base_prices[ticker] = loadStooqData(ticker,start)

for ticker in yahoo_tickers:
    base_prices[ticker] = loadYahooData(ticker,start)



In [20]:
base_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2978 entries, 2012-01-03 to 2023-07-21
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   XAUPLN    2978 non-null   float64
 1   ^TBSP     2882 non-null   float64
 2   MWIG40TR  2882 non-null   float64
 3   USDPLN    2977 non-null   float64
 4   EURPLN    2977 non-null   float64
 5   ISAC.L    2915 non-null   float64
 6   CORP.L    2733 non-null   float64
dtypes: float64(7)
memory usage: 186.1+ KB


* load data
* describe data in local currency
* transform data do pln
* describe data in pln

In [16]:
gold = loadStooqData('XAUPLN',start)